In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hatenorm23/sample_submission.csv
/kaggle/input/hatenorm23/train.csv
/kaggle/input/hatenorm23/test.csv


In [2]:
df_train = pd.read_csv('/kaggle/input/hatenorm23/train.csv', sep='|')
df_test = pd.read_csv('/kaggle/input/hatenorm23/test.csv', sep='|')

In [3]:
df_train

,Id,sentence,span,bio
0,0,"Say it loud , say it clear , illegal #immigran...","{'start': [8, 9], 'end': [8, 9]}",O O O O O O O O B I O O O O O
1,1,"Islam is not a religion, its just an excuse fo...","{'start': [0, 10], 'end': [4, 13]}",B I I I I O O O O O B I I I O O O O
2,2,@user Care About Illegals Breaking U.S #Immigr...,"{'start': [40], 'end': [40]}",O O O O O O O O O O O O O O O O O O O O O O O ...
3,3,Behold the future DACA America - unless we Man...,"{'start': [10, 17], 'end': [14, 18]}",O O O O O O O O O O B I I I I O O B I O O O O ...
4,4,No that little tattoo of a flower on your ankl...,"{'start': [14, 20, 25, 26], 'end': [14, 21, 25...",O O O O O O O O O O O O O O B O O O O O B I O ...
...,...,...,...,...
2416,2416,but if you are a nigger you can beat rape your...,"{'start': [5, 8], 'end': [5, 9]}",O O O O O B O O B I O O O O O O O O O O O O O ...
2417,2417,Immigrants Come LEGALLY like 1.5 Milion Do Eve...,"{'start': [0, 2, 13, 20], 'end': [0, 2, 13, 20]}",B O B O O O O O O O O O O B O O O O O O B O O ...
2418,2418,lol kike,"{'start': [1], 'end': [1]}",O B
2419,2419,Our culture elements must be learned by the Mu...,"{'start': [8, 22], 'end': [8, 25]}",O O O O O O O O B O O O O O O O O O O O O O B ...


In [4]:
mapping_to_int = {
    'B': 0,
    'I': 1,
    'O': 2
}

In [5]:
bios = df_train['bio'].to_list()
for i in range(len(bios)):
    bio = bios[i]
    for k in mapping_to_int.keys():
        bio = bio.replace(k, str(mapping_to_int[k]))
    bio = bio.split()
    for j in range(len(bio)):
        bio[j] = int(bio[j])
    bios[i] = bio
df_train['ner_tags'] = bios

In [6]:
sentences = df_train['sentence'].to_list()
for i in range(len(sentences)):
    sentence = sentences[i]
    sentence = sentence.split()
    sentences[i] = sentence
df_train['tokens'] = sentences

In [7]:
for i, j in zip(df_train['tokens'].to_list(), df_train['ner_tags'].to_list()):
    if len(i) != len(j):
        print(i,j)

In [8]:
from sklearn.model_selection import train_test_split
df_train, df_eval = train_test_split(df_train, test_size=0.2, random_state=42)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [9]:
df_train = df_train[['Id', 'ner_tags', 'tokens']]
df_eval = df_eval[['Id', 'ner_tags', 'tokens']]

In [10]:
df_train

,Id,ner_tags,tokens
807,807,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, ...","[@user, @user, TRAVEL, ADVISORYDominic, Durden..."
281,281,"[2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, ...","[Wow, ,, Stupid, is, as, Stupidity, does, Thes..."
1463,1463,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, ...","[@user, @user, ,, you, don't, have, to, be, a,..."
1764,1764,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, ...","[the, fucking, kike, sob, banned, my, favorite..."
1746,1746,"[2, 2, 2, 2, 2, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, ...","[The, British, politicians, want, to, impose, ..."
...,...,...,...
1638,1638,"[0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]","[Jews, are, not, only, insane, ,, they, are, o..."
1095,1095,"[0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 1, 2, 2, 2, ...","[Rape, charges, isnt, the, way, to, get, reven..."
1130,1130,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[Quite, strange,, the, majority, of, Easter, E..."
1294,1294,"[2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, ...","[Was, getting, elbowed, by, a, triple, chinned..."


In [11]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(df_train)
eval_dataset = Dataset.from_pandas(df_eval)

In [12]:
train_dataset[0]

{'Id': 807,
 'ner_tags': [2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  0,
  1,
  2,
  2,
  2,
  0,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2],
 'tokens': ['@user',
  '@user',
  'TRAVEL',
  'ADVISORYDominic',
  'Durden',
  '&',
  'Mollie',
  'Tibbetts',
  '&',
  'Kate',
  'Steinle',
  'were',
  'murdered',
  'by',
  'Illegal',
  'Aliens',
  '!',
  'We',
  'must',
  '#BuildThatWallFollow',
  '@user',
  '&',
  '@user',
  '&',
  '#RETWEET',
  'Help',
  'out',
  '&',
  'order',
  'bricks',
  'or',
  'donate',
  'today',
  'at',
  'Follow',
  '@user'],
 '__index_level_0__': 807}

In [13]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [14]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [15]:
tokenized_train_dataset = train_dataset.map(tokenize_and_align_labels, batched=True)
tokenized_eval_dataset = eval_dataset.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [16]:
tokenized_train_dataset

Dataset({
    features: ['Id', 'ner_tags', 'tokens', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1936
})

In [17]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [18]:
!pip install evaluate -q
!pip install seqeval -q
import evaluate
seqeval = evaluate.load("seqeval")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [19]:
import numpy as np

labels = ['B', 'I', 'O']

label_list = ['B', 'I', 'O']

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [20]:
id2label = {
    0 : 'B',
    1 : 'I',
    2 : 'O'
}
label2id = {
    'B': 0,
    'I': 1,
    'O': 2
}

In [21]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=3, id2label=id2label, label2id=label2id
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream

In [22]:
training_args = TrainingArguments(
    output_dir="model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to = "none"
)

In [23]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [24]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.382397,0.324698,0.227879,0.267806,0.856778
2,No log,0.346230,0.393310,0.413333,0.403073,0.873816


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=122, training_loss=0.4281122489053695, metrics={'train_runtime': 31.1629, 'train_samples_per_second': 124.25, 'train_steps_per_second': 3.915, 'total_flos': 69394382467200.0, 'train_loss': 0.4281122489053695, 'epoch': 2.0})

In [25]:
from transformers import pipeline

In [26]:
classifier = pipeline("ner", model="/kaggle/working/model/checkpoint-122")

In [27]:
df_test.head(2)

,Id,sentence
0,0,"Has the penny , or should I say peseta , final..."
1,1,it painfully clear why mudslime throw these pe...


In [28]:
sentences = df_test['sentence'].to_list()
for i in range(len(sentences)):
    sentence = sentences[i]
    sentence = sentence.split()
    sentences[i] = sentence
df_test['tokens'] = sentences
text = df_test['tokens'].to_list()

In [29]:
df_test

,Id,sentence,tokens
0,0,"Has the penny , or should I say peseta , final...","[Has, the, penny, ,, or, should, I, say, peset..."
1,1,it painfully clear why mudslime throw these pe...,"[it, painfully, clear, why, mudslime, throw, t..."
2,2,says the kike cunt trying to charge for premiu...,"[says, the, kike, cunt, trying, to, charge, fo..."
3,3,Well The Hoes might pull this off . Pardon the...,"[Well, The, Hoes, might, pull, this, off, ., P..."
4,4,All of our tim hortons are staffed by those pe...,"[All, of, our, tim, hortons, are, staffed, by,..."
...,...,...,...
601,601,that is average height for women you niggas ju...,"[that, is, average, height, for, women, you, n..."
602,602,Democrats Are Not On Americas SideThey Want Op...,"[Democrats, Are, Not, On, Americas, SideThey, ..."
603,603,This is our import for the last 30 years: musl...,"[This, is, our, import, for, the, last, 30, ye..."
604,604,Karma will get you @user u kunt #mkr,"[Karma, will, get, you, @user, u, kunt, #mkr]"


In [30]:
# This returned subword level classifications which are not useful for us
# TODO: Find way to map back

# from transformers import AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained("/kaggle/working/model/checkpoint-122")
# from transformers import AutoModelForTokenClassification
# import torch
# model = AutoModelForTokenClassification.from_pretrained("/kaggle/working/model/checkpoint-122")

# predicted_token_classes = []
# for i in range(len(text)):
#     sample = text[i]
#     inputs = tokenizer(sample, return_tensors="pt", is_split_into_words=True)
    
#     with torch.no_grad():
#         logits = model(**inputs).logits
#     predictions = torch.argmax(logits, dim=2)
#     for i in range(len(predictions)):
#         preds = []
#         for t in predictions[i]:
#             preds.append(model.config.id2label[t.item()])
#         predicted_token_classes.append(preds)
        
# predicted_token_classes_as_string = []
# for i in predicted_token_classes:
#     predicted_token_classes_as_string.append(" ".join(i))
    
# df_test['bio'] = predicted_token_classes_as_string
# df_submission = df_test[['Id', 'bio']]
# df_submission.to_csv('sub.csv', index=False)

In [31]:
from transformers import pipeline
from tqdm import trange
classifier = pipeline("ner", model="/kaggle/working/model/checkpoint-122")

ls = []
for i in trange(len(text)):
    t = classifier(text[i])
    ls.append(t)

100%|██████████| 606/606 [05:42<00:00,  1.77it/s]


In [32]:
outputs = []
for pred in ls:
    output = []
    for i in pred:
        if len(i) == 0:
            output.append('O')
        else:
            output.append(i[0]['entity'])
    outputs.append(output)

In [33]:
for i in range(len(outputs)):
    outputs[i] = " ".join(outputs[i])

In [34]:
df_test['bio'] = outputs
df_submission = df_test[['Id', 'bio']]
df_submission.to_csv('sub.csv', index=False)